In [1]:
from PIL import Image
import numpy as np
import time
import random as rm
import math

# Below all directly copied from https://github.com/huggingface/pytorch-pretrained-BigGAN
import torch
from pytorch_pretrained_biggan import (BigGAN, one_hot_from_int, one_hot_from_names, truncated_noise_sample,
                                       save_as_images, display_in_terminal)

# Load pre-trained model tokenizer (vocabulary)
model = BigGAN.from_pretrained('biggan-deep-512')

In [5]:
# RUN BEFORE EVERY CYCLE STARTED (regenerate initial position every time)
# initial position (first child and mating parents)

# truncation originally 0.4
truncation = 0.4

# create randomly initialised location vectors for main parent ('child') and mating parents
location_vectors = one_hot_from_names('tench', batch_size=4)
for i in range(4):
    for j in range(1000):
        location_vectors[i][j] = np.random.uniform(-0.2,0.2)

# Prepare 1 main parent and 3 potential mating parents
noise_vectors = truncated_noise_sample(truncation=truncation, batch_size=4)
for i in range(4):
    for j in range(128):
        noise_vectors[i][j] = 0.

# All in tensors
noise_vectors = torch.from_numpy(noise_vectors)
location_vectors = torch.from_numpy(location_vectors)

# Generate an image
with torch.no_grad():
    output = model(noise_vectors, location_vectors, truncation)

# Save results as png images
save_as_images(output)

In [ ]:
# looping evolution while saving in between images NOISE TO 0-VECTOR

while True:
    # prompt user choice mating partner
    inp = 0
    while not inp in ["1","2","3","s"]:
        inp = input("Mate with picture [1,2,3] ")
    if inp == "s":
        break
    
    child_vector = (location_vectors[0] + location_vectors[int(inp)])/2
    
    # create images between current and child vector
    in_betweens = one_hot_from_names('tench', batch_size=5)
    for i in range(5):
        in_betweens[i] = (location_vectors[0]*(6-i) + location_vectors[int(inp)])/(7-i)
        print('(',6-i, '+ 1 )/', 7-i)
    print(in_betweens)
    
    #UGLY, FIX LATER!! (Maybe okay to just skip, check types through runs)
    # solve that child vector sometimes not accepted (if tensor instead of numpy array)
    print(type(child_vector))
    print(type(location_vectors))
    if type(child_vector) == type(location_vectors):
        child_vector = child_vector.numpy()
        print(child_vector)
        
    parent_vectors = one_hot_from_names('tench', batch_size=3)
    for i in range(3):
        for j in range(1000):
            parent_vectors[i][j] = np.random.uniform(-0.2,0.2)
    
    # combine child, parents and in between images into class vectors (numpy array)
    location_vectors = np.append([child_vector], parent_vectors, axis=0)
    location_vectors = np.append(location_vectors, in_betweens, axis=0)
    
    noise_vectors = truncated_noise_sample(truncation=truncation, batch_size=9)
    for i in range(9):
        for j in range(128):
            noise_vectors[i][j] = 0.

    # All in tensors
    noise_vectors = torch.from_numpy(noise_vectors)
    location_vectors = torch.from_numpy(location_vectors)

    # Generate an image
    with torch.no_grad():
        output = model(noise_vectors, location_vectors, truncation)

    # Save results as png images
    save_as_images(output)

Mate with picture [1,2,3] 2
( 6 + 1 )/ 7
( 5 + 1 )/ 6
( 4 + 1 )/ 5
( 3 + 1 )/ 4
( 2 + 1 )/ 3
[[-0.1305734   0.01696502 -0.06074684 ...  0.13399938  0.17174543
   0.14021568]
 [-0.12904797  0.01769428 -0.0636666  ...  0.13324833  0.1708594
   0.13388427]
 [-0.12691234  0.01871524 -0.06775427 ...  0.13219687  0.16961893
   0.1250203 ]
 [-0.12370891  0.02024667 -0.07388578 ...  0.1306197   0.16775826
   0.11172431]
 [-0.11836985  0.02279907 -0.08410496 ...  0.12799107  0.16465713
   0.08956435]]
<class 'torch.Tensor'>
<class 'torch.Tensor'>
[-1.07691735e-01  2.79038511e-02 -1.04543313e-01 -1.15741007e-02
  2.44561173e-02  1.43341318e-01 -1.42222583e-01 -8.87795016e-02
  1.00335985e-01  2.03241408e-03 -3.33382674e-02  1.22739017e-01
 -1.50377482e-01  1.19032115e-02  4.01485860e-02  6.90203831e-02
 -1.70370102e-01  6.78530037e-02 -7.47772902e-02 -4.67119701e-02
 -1.36603594e-01 -7.25957677e-02  1.62440762e-02 -8.66007358e-02
 -9.36897919e-02 -3.91828418e-02 -1.41955093e-01 -4.25561294e-02
 

In [4]:
# calculate distance between vector points
distance_00 = math.sqrt(sum(location_vectors[0] - location_vectors[0])**2)
distance_01 = math.sqrt(sum(location_vectors[0] - location_vectors[1])**2)
distance_02 = math.sqrt(sum(location_vectors[0] - location_vectors[2])**2)
distance_03 = math.sqrt(sum(location_vectors[0] - location_vectors[3])**2)
print(distance_00)
print(distance_01)
print(distance_02)
print(distance_03)

# range observed so far: 0.0 (self) to 4.2, 7.7!

## Find way to set a distance, generate vectors within that range (or on that exact distance?)
# hmmmmmm, each axis value randomly generated
# 1) Keep each axis between certain range 
# (but then very limited in movement, distance set way bigger than actual distance obtained)
# 2) Randomly pick axes to fill, keep summing distance total to check if continue 
# (so certain axes will not be changed, (is that a problem?) but freedom within 'chosen' changes)

## Problem how to interpolate between pictures if noise also changes 
# 1) Keep same noise for child and potential parents? (and thus also for in between images)
# (always same noise -> problem?)
# 2) Change chosen mating parent noise vector to childs noise vector (because never end up there anyway), intermediate images same noise vector
# (When change child's noise vector? Weird to keep same noise vector through whole process)
# 3) Some combination?

## Problem: still not 'exact' vector pointer: how child and noise combined?!

0.0
3.9867687738753617
1.4651014584506006
2.4868623769343414
